In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('agg-config.txt')


time_step = int(config.get('Configuration', 'time_step'))
time_step2 = int(config.get('Configuration', 'time_step2'))

window_size = int(config.get('Configuration', 'window_size'))
processed_data_folder = config.get('Configuration', 'processed_data_folder')
raw_data_folder = config.get('Configuration', 'raw_data_folder')

ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')


activities=config.get('Configuration', 'learn_activities')
activities = [(item.strip()) for item in activities.split(',')]

cross_days=config.get('Configuration', 'cross_days')
cross_days = [(item.strip()) for item in cross_days.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))

users_day={}
for cross_day in cross_days:
    users_day[cross_day]=config.get('Configuration', cross_day)
    users_day[cross_day] = [(item.strip()) for item in users_day[cross_day].split(',')]
print(users_day)

{'2024-07-05': ['0001', '3d57', 'ed9c'], '2024-07-06': ['0001', '3d57', 'ed9c'], '2024-07-07': ['0001', '3d57', 'ed9c'], '2024-07-18': ['16fe', 'f02e', 'ed9c'], '2024-07-19': ['16fe', 'f02e', 'ed9c'], '2024-07-20': ['16fe', 'f02e', 'ed9c'], '2024-07-24': ['16fe', '5b66', 'ed9c'], '2024-07-25': ['16fe', '5b66', 'ed9c'], '2024-07-26': ['16fe', '5b66', 'ed9c'], '2024-07-27': ['16fe', '5b66', 'ed9c'], '2024-07-28': ['16fe', '5b66', 'ed9c'], '2024-07-29': ['16fe', '5b66', 'ed9c'], '2024-07-30': ['16fe', '5b66', 'ed9c'], '2024-07-31': ['16fe', '5b66', 'ed9c']}


In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

def sizeT():
    return relT(tN) 


def initTime(ini_date,end_date):
    tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
    print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))

    print(ini_date)
    t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
    print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

    day=list(range(day_time(t0),day_time(tN)+1))[0]
    return day,t0,tN
    
initTime(ini_date,end_date)


from datetime import datetime, timedelta

def getStrDatefrom(day_number):
    # Reference date (1970-01-01)
    reference_date = datetime(1970, 1, 1)
    
    # Compute the date corresponding to the day number
    resulting_date = reference_date + timedelta(days=day_number)
    
    return resulting_date.strftime("%Y-%m-%d")


2024-08-01 01:59:59 End date is 1722470399 day: 19936 2024-08-01 01:59:59
2024-07-31 02:00:00
2024-07-31 02:00:00 Init date is 1722384000 day: 19935 2024-07-31 02:00:00


In [3]:
import os
from PIL import Image 
import cv2

def getGTImg(day,user,tt):
    file_path = os.path.join(processed_data_folder+"DAY_"+str(day)+"/"+user+"_agg/FP."+str(tt)+".png")
    if os.path.exists(file_path):
        #print(file_path,"OK")
        return rescale_image(np.array(Image.open(file_path)),8.0) / 255.0
    else:
        #print(file_path,"KO")
        return None
    
def getAREAactivity0(day,activity,user):
    file_path = os.path.join(processed_data_folder+"DAY_"+str(day)+"/"+user+"."+str(activity)+".png")
    print(file_path)
    #file_path = os.path.join(processed_data_folder+"DAY_"+str(day)+"/"+user+"_agg/FP."+str(tt)+".png")
    if os.path.exists(file_path):
        return rescale_image(np.array(Image.open(file_path)),1.0) / 255.0
    else:
        return np.zeros((h*8,w*8))
    
def getAREAactivity(activity,user):
    file_path = os.path.join(processed_data_folder+"/mask_loc_act/"+str(activity)+".png")
    if os.path.exists(file_path):
        return np.array(Image.open(file_path)) / 255.0
    else:
        file_path = os.path.join(processed_data_folder+"/mask_loc_act/"+str(user)+"."+activity+".png")
        if os.path.exists(file_path):
            return np.array(Image.open(file_path)) / 255.0
        print("ERROR MASK ACTIVITY",activity,user)
        return None
    
def rescale_image(image, scale_factor):
    width = int(image.shape[1] * scale_factor)
    height = int(image.shape[0] * scale_factor)
    dimensions = (width, height)
    rescaled_image = cv2.resize(image, dimensions, interpolation=cv2.INTER_LINEAR)
    return rescaled_image

In [4]:
from datetime import datetime, timedelta

days=[]
t0N_days={}

for cross_day in cross_days:
    print(cross_day)
    # Convertir las fechas a objetos datetime
    ini_date_dt = datetime.strptime(ini_date, "%Y-%m-%d %H:%M:%S")
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")
    cross_day_dt = datetime.strptime(cross_day, "%Y-%m-%d")

    # Reemplazar el día manteniendo la hora original
    new_ini_date = cross_day_dt.replace(hour=ini_date_dt.hour, minute=ini_date_dt.minute, second=ini_date_dt.second)
    new_end_date = cross_day_dt.replace(hour=end_date_dt.hour, minute=end_date_dt.minute, second=end_date_dt.second) + timedelta(days=1)

    # Convertir las fechas de nuevo a cadena si es necesario
    new_ini_date_str = new_ini_date.strftime("%Y-%m-%d %H:%M:%S")
    new_end_date_str = new_end_date.strftime("%Y-%m-%d %H:%M:%S")

    print(f"New ini_date: {new_ini_date_str}")
    print(f"New end_date: {new_end_date_str}")
    day,t0,tN=initTime(new_ini_date_str,new_end_date_str)
    days.append(day)
    t0N_days[day]=(t0,tN)
    

print(days)
print(t0N_days)



2024-07-05
New ini_date: 2024-07-05 02:00:00
New end_date: 2024-07-06 01:59:59
2024-07-06 01:59:59 End date is 1720223999 day: 19910 2024-07-06 01:59:59
2024-07-05 02:00:00
2024-07-05 02:00:00 Init date is 1720137600 day: 19909 2024-07-05 02:00:00
2024-07-06
New ini_date: 2024-07-06 02:00:00
New end_date: 2024-07-07 01:59:59
2024-07-07 01:59:59 End date is 1720310399 day: 19911 2024-07-07 01:59:59
2024-07-06 02:00:00
2024-07-06 02:00:00 Init date is 1720224000 day: 19910 2024-07-06 02:00:00
2024-07-07
New ini_date: 2024-07-07 02:00:00
New end_date: 2024-07-08 01:59:59
2024-07-08 01:59:59 End date is 1720396799 day: 19912 2024-07-08 01:59:59
2024-07-07 02:00:00
2024-07-07 02:00:00 Init date is 1720310400 day: 19911 2024-07-07 02:00:00
2024-07-18
New ini_date: 2024-07-18 02:00:00
New end_date: 2024-07-19 01:59:59
2024-07-19 01:59:59 End date is 1721347199 day: 19923 2024-07-19 01:59:59
2024-07-18 02:00:00
2024-07-18 02:00:00 Init date is 1721260800 day: 19922 2024-07-18 02:00:00
2024-07-

In [5]:
import os




for day in days:
    other_days= [other for other in days if other != day]
    
    day_str=getStrDatefrom(day)
    users=users_day[day_str]
    print(day,other_days,users)
    for activity in activities:
        print("\t",activity)

        for user in users:
            print("\t",user)
            os.makedirs(processed_data_folder+"/DAY_"+str(day)+"/act/", exist_ok=True)

            with open(processed_data_folder+"/DAY_"+str(day)+"/act/loc."+user+"."+activity+".tsv", "w") as file:    

                zone_act=np.zeros((h*8,w*8))
                for other_day in other_days:
                    zone_act=zone_act+getAREAactivity0(other_day,activity,user)
                zone_act=zone_act/np.max(zone_act)
                # Ajustar los valores: poner a 0 los menores de 0.5 y escalar los valores entre 0.5 y 1
                zone_act = np.where(zone_act < 0.5, 0, zone_act)  # Establecer a 0 los valores menores de 0.5
                zone_act = np.where((zone_act >= 0.5) & (zone_act <= 1), (zone_act - 0.5) / 0.5, zone_act)  # Escalar valores entre 0.5 y 1
                
                cv2.imwrite(processed_data_folder+"/DAY_"+str(day)+"/other."+user+"."+activity+".png", rescale_image(zone_act,1)*255.0) 

                zone_act=getAREAactivity(activity,user)
                
                lastImg=np.zeros((h*8,w*8))
                (t0,tN)=t0N_days[day]
                print("\t\t\t",t0,tN)
                for tt in range(t0,tN,time_step2):

                    imgLocation=getGTImg(day,user,tt)
                    if(imgLocation is None):
                        imgLocation=lastImg

                    A_min = np.min(imgLocation)
                    A_max = np.max(imgLocation)
                    if(A_max<0.05):
                        print("\t zero image!",tt)
                        imgLocation=lastImg
                    else:
                        lastImg=imgLocation
                        imgLocation = (imgLocation - A_min) / (A_max - A_min)

                    #print(user,tt,A_min,A_max,imgAreaActivity.shape,imgLocation.shape)

                    if imgLocation is not None:
                        degree = np.max(np.minimum(imgLocation, zone_act))
                        #print(degree)
                        timestamp=pd.to_datetime(time2str(tt))
                        file.write(str(timestamp)+"\t"+str(degree)+"\n")
                    else:
                        file.write(str(timestamp)+"\t"+str(degree)+"\n")
    


19909 [19910, 19911, 19922, 19923, 19924, 19928, 19929, 19930, 19931, 19932, 19933, 19934, 19935] ['0001', '3d57', 'ed9c']
	 toileting
	 0001
./data-har/DAY_19910/0001.toileting.png
./data-har/DAY_19911/0001.toileting.png
./data-har/DAY_19922/0001.toileting.png
./data-har/DAY_19923/0001.toileting.png
./data-har/DAY_19924/0001.toileting.png
./data-har/DAY_19928/0001.toileting.png
./data-har/DAY_19929/0001.toileting.png
./data-har/DAY_19930/0001.toileting.png
./data-har/DAY_19931/0001.toileting.png
./data-har/DAY_19932/0001.toileting.png
./data-har/DAY_19933/0001.toileting.png
./data-har/DAY_19934/0001.toileting.png
./data-har/DAY_19935/0001.toileting.png
			 1720137600 1720223999
	 zero image! 1720137600
	 3d57
./data-har/DAY_19910/3d57.toileting.png
./data-har/DAY_19911/3d57.toileting.png
./data-har/DAY_19922/3d57.toileting.png
./data-har/DAY_19923/3d57.toileting.png
./data-har/DAY_19924/3d57.toileting.png
./data-har/DAY_19928/3d57.toileting.png
./data-har/DAY_19929/3d57.toileting.png


/tmp/ipykernel_14549/264077917.py:24: RuntimeWarning: invalid value encountered in divide
  zone_act=zone_act/np.max(zone_act)


	 zero image! 1720144080
	 zero image! 1720144140
	 zero image! 1720144200
	 zero image! 1720144260
	 zero image! 1720144320
	 zero image! 1720144380
	 zero image! 1720144440
	 zero image! 1720144500
	 ed9c
./data-har/DAY_19910/ed9c.pc.png
./data-har/DAY_19911/ed9c.pc.png
./data-har/DAY_19922/ed9c.pc.png
./data-har/DAY_19923/ed9c.pc.png
./data-har/DAY_19924/ed9c.pc.png
./data-har/DAY_19928/ed9c.pc.png
./data-har/DAY_19929/ed9c.pc.png
./data-har/DAY_19930/ed9c.pc.png
./data-har/DAY_19931/ed9c.pc.png
./data-har/DAY_19932/ed9c.pc.png
./data-har/DAY_19933/ed9c.pc.png
./data-har/DAY_19934/ed9c.pc.png
./data-har/DAY_19935/ed9c.pc.png
			 1720137600 1720223999
	 zero image! 1720137600
	 zero image! 1720169880
	 sleep
	 0001
./data-har/DAY_19910/0001.sleep.png
./data-har/DAY_19911/0001.sleep.png
./data-har/DAY_19922/0001.sleep.png
./data-har/DAY_19923/0001.sleep.png
./data-har/DAY_19924/0001.sleep.png
./data-har/DAY_19928/0001.sleep.png
./data-har/DAY_19929/0001.sleep.png
./data-har/DAY_19930/

	 zero image! 1721961000
	 zero image! 1721961060
	 zero image! 1721962440
	 zero image! 1721962500
	 zero image! 1721962560
	 zero image! 1721962740
	 zero image! 1721962800
	 zero image! 1721962920
	 zero image! 1721962980
	 zero image! 1721963100
	 zero image! 1721963220


	 ed9c
			 1721952000 1722038399
	 zero image! 1721952000


...

	 zero image! 1722417060
	 zero image! 1722417120
	 zero image! 1722417180
	 zero image! 1722417360
	 zero image! 1722417420
	 zero image! 1722417480
	 zero image! 1722417900
	 zero image! 1722418200


	 kitchen
	 16fe
			 1722384000 1722470399
	 zero image! 1722384000


	 5b66
			 1722384000 1722470399
	 zero image! 1722384000


	 ed9c
			 1722384000 1722470399
	 zero image! 1722384000


	 zero image! 1722417060
	 zero image! 1722417120
	 zero image! 1722417180
	 zero image! 1722417360
	 zero image! 1722417420
	 zero image! 1722417480
	 zero image! 1722417900
	 zero image! 1722418200
